In [1]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from scipy.sparse import csr_matrix
import sparse_dot_topn.sparse_dot_topn as ct


In [2]:

failed =  pd.read_csv('/home/sid/development/python/music-analysis/Temp Sheets/Failed Results (Union minus Merged).csv')

failed = failed.drop_duplicates()

In [3]:
failed

,artist,track,datetime,album,release_date,genres,length,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,Declan Mckenna,In Blue,2020-08-25 11:10:56,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Declan Mckenna,In Blue,2020-08-07 22:31:03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Declan Mckenna,In Blue,2020-03-27 23:02:03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Everything Everything,Don't Let It Bring You Down - Live At Bbc Maid...,2020-08-24 13:17:44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Gotham Knights,World Premiere Trailer,2020-08-24 10:37:20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17312,Various Artists,Überflieger,NaN,MrSuicideSheep Presents - Taking You Higher,2015-10-09,[],530750.0,25.0,0.649,0.864,10.0,-6.659,0.0,0.0497,0.0172,0.864000,0.0308,0.5380,127.987,4.0
17313,Various Artists,Slumber Party,NaN,MrSuicideSheep Presents - Taking You Higher,2015-10-09,[],233587.0,37.0,0.533,0.936,1.0,-4.626,1.0,0.0447,0.0143,0.721000,0.0622,0.4210,128.018,4.0
17314,Various Artists,Ragtime Cat (feat. Lilja Bloom),NaN,"Electro Swing Fever, Vol. 3",2014-02-17,[],181000.0,22.0,0.736,0.715,5.0,-4.114,0.0,0.0412,0.0120,0.000796,0.2180,0.8510,119.976,4.0
17315,Yoe Mase,Nothing More (deaf Kev Remix),NaN,Nothing More (Deaf Kev Remix),2015-11-06,"['chillstep', 'pop edm']",376200.0,37.0,0.383,0.831,1.0,-5.063,1.0,0.0511,0.0123,0.066100,0.1020,0.0912,199.850,4.0


In [4]:
def ngrams(string, n=3):
    string = re.sub(r'[,-./]|\sBD',r'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]


In [5]:
tracks = failed['track']
vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
tf_idf_matrix = vectorizer.fit_transform(tracks)

In [6]:
def awesome_cossim_top(A, B, ntop, lower_bound=0):
    # force A and B as a CSR matrix.
    # If they have already been CSR, there is no overhead
    A = A.tocsr()
    B = B.tocsr()
    M, _ = A.shape
    _, N = B.shape
 
    idx_dtype = np.int32
 
    nnz_max = M*ntop
 
    indptr = np.zeros(M+1, dtype=idx_dtype)
    indices = np.zeros(nnz_max, dtype=idx_dtype)
    data = np.zeros(nnz_max, dtype=A.dtype)

    ct.sparse_dot_topn(
        M, N, np.asarray(A.indptr, dtype=idx_dtype),
        np.asarray(A.indices, dtype=idx_dtype),
        A.data,
        np.asarray(B.indptr, dtype=idx_dtype),
        np.asarray(B.indices, dtype=idx_dtype),
        B.data,
        ntop,
        lower_bound,
        indptr, indices, data)

    return csr_matrix((data,indices,indptr),shape=(M,N))


In [7]:
import time
t1 = time.time()
matches = awesome_cossim_top(tf_idf_matrix, tf_idf_matrix.transpose(), 10, 0.8)
t = time.time()-t1
print("SELFTIMED:", t)

SELFTIMED: 0.5143001079559326


In [8]:
def get_matches_df(sparse_matrix, name_vector, top=100):
    non_zeros = sparse_matrix.nonzero()
    
    sparserows = non_zeros[0]
    sparsecols = non_zeros[1]
    
    if top:
        nr_matches = top
    else:
        nr_matches = sparsecols.size
    
    left_side = np.empty([nr_matches], dtype=object)
    right_side = np.empty([nr_matches], dtype=object)
    similairity = np.zeros(nr_matches)
    
    for index in range(0, nr_matches):
        left_side[index] = name_vector[sparserows[index]]
        right_side[index] = name_vector[sparsecols[index]]
        similairity[index] = sparse_matrix.data[index]
    
    return pd.DataFrame({'left_side': left_side,
                          'right_side': right_side,
                           'similarity': similairity})
        

In [9]:
matches_df = get_matches_df(matches, tracks, top=100000)
# matches_df = matches_df[matches_df['similarity'] < 0.99999] # Remove all exact matches

In [10]:
matches_df.sort_values(by='similarity', ascending=False).to_csv('bergvca Matches.csv', index=False)